In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from scipy.linalg import hankel
from sklearn.decomposition import PCA
from tensorflow.keras import regularizers



from keras.layers import LSTM, Dense

# Function to create lag features and rolling statistics
def create_lag_features(data, lags):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data['Close'].pct_change(lag)  # Percentage change
    data['rolling_mean'] = data['Close'].rolling(window=5).mean()
    data['rolling_std'] = data['Close'].rolling(window=5).std()
    return data

# Function to create Henkel matrix from the time series data
def create_henkel_matrix(data, window_size):
    """Create Henkel matrix from time series data."""
    series = data['Close'].values
    hankel_matrix = hankel(series[:window_size], series[window_size-1:])
    return hankel_matrix

def decompose_henkel_matrix(hankel_matrix):
    """Perform Singular Value Decomposition (SVD) to extract components."""
    u, s, vh = np.linalg.svd(hankel_matrix)
    
    # Make sure to reshape s into a diagonal matrix
    s_diag = np.diag(s)
    
    # Ensure correct dimensions by slicing u and vh accordingly
    trend = u[:, 0:1] @ s_diag[0:1, 0:1] @ vh[0:1, :]  # Trend from first component
    seasonal = u[:, 1:3] @ s_diag[1:3, 1:3] @ vh[1:3, :]  # Seasonal components
    noise = hankel_matrix - (trend + seasonal)  # Remaining noise

    return trend, seasonal, noise


# Function to perform quantum-inspired SVD (QSVD)
def quantum_svd(hankel_matrix, num_components=3):
    """Quantum-inspired approximation for SVD."""
    u, s, vh = np.linalg.svd(hankel_matrix)
    return u[:, :num_components], s[:num_components], vh[:num_components, :]
def split_data(df, training_period, train_ratio=0.8):
    # Exclude the 'Date' column from features during scaling
    df = create_lag_features(df, training_period)
    df.dropna(inplace=True)  # Remove rows with NaN values

    # Drop the 'Date' column and then scale the features
    feature_columns = df.drop(columns=['Date', 'Close']).columns
    feature_scaler = StandardScaler()
    scaled_features = feature_scaler.fit_transform(df[feature_columns])

    # Re-create the dataframe with the scaled features
    scaled_data = pd.DataFrame(scaled_features, columns=feature_columns, index=df.index)
    scaled_data['Close'] = df['Close']  # Keep original target for splitting

    # Scale the target variable (Close price)
    target_scaler = StandardScaler()
    scaled_data['Close'] = target_scaler.fit_transform(df['Close'].values.reshape(-1, 1)).ravel()

    # Split data
    split_idx = int(len(df) * train_ratio)
    train_data = scaled_data.iloc[:split_idx]
    test_data = scaled_data.iloc[split_idx:]

    # Separate features and targets
    X_train = train_data.drop(columns=['Close'])
    y_train = train_data['Close']
    X_test = test_data.drop(columns=['Close'])
    y_test = test_data['Close']

    return X_train, y_train, X_test, y_test, test_data.index, feature_scaler, target_scaler


# Define the CNN model
def train_cnn(X_train, y_train, input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50, return_sequences=False))  # Add GRU layer
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.005), loss='mean_squared_error')

    # Reshape input data for CNN
    X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))  # Add channel dimension

    # Reduce learning rate on plateau callback
    lr_scheduler = ReduceLROnPlateau(
        monitor='loss',
        factor=0.8,
        patience=5,
        verbose=1,
        min_lr=1e-6
    )

    # Train the model
    model.fit(
        X_train, y_train.values,
        epochs=50,
        batch_size=32,
        verbose=1,
        callbacks=[lr_scheduler]
    )

    return model

# Function to calculate log-return volatility
def calculate_log_volatility(y_values):
    log_returns = np.log(y_values[1:] / y_values[:-1])  # Logarithmic returns
    return np.std(log_returns)  # Volatility as standard deviation of log returns

# Updated evaluate function with average MSE in the legend
def evaluate_multiple_start_points(model, X_test, y_test, test_index, n_days_to_predict, num_start_points, feature_scaler, target_scaler):
    errors = []
    volatilities = []

    # Select start indices
    start_indices = range(0, 30, 5)  # Example: 5 starting points, spaced 5 indices apart

    for start_idx in start_indices:
        end_idx = start_idx + n_days_to_predict

        # Ensure the model always gets the past 15 days of data
        context_idx = start_idx - 15
        X_context = X_test.iloc[context_idx:start_idx]  # Past 15 days
        
        X_sub = X_test.iloc[start_idx:end_idx]
        y_true = y_test.iloc[start_idx:end_idx]

        # Predict and invert scaling for the target
        predictions = model.predict(X_sub)
        predictions = target_scaler.inverse_transform(predictions.reshape(-1, 1)).ravel()
        y_true_unscaled = target_scaler.inverse_transform(y_true.values.reshape(-1, 1)).ravel()

        # Calculate MSE
        mse = mean_squared_error(y_true_unscaled, predictions)
        errors.append(mse)

        # Calculate log-return volatility
        log_volatility = calculate_log_volatility(y_true_unscaled)
        volatilities.append(log_volatility)

        # Plot individual predictions
        plt.figure(figsize=(10, 6))
        plt.plot(test_index[start_idx:end_idx], y_true_unscaled, label="True Data", color="green")
        plt.plot(test_index[start_idx:end_idx], predictions, label="Predictions", color="orange")
        plt.xlabel("Date")
        plt.ylabel("Close Price")
        plt.title(f"Predictions from Index {start_idx} to {end_idx}")
        plt.legend()
        os.makedirs("../TSLA_pred/cnn/", exist_ok=True)
        plt.savefig(f"../TSLA_pred/cnn/start_{start_idx}.png")
        plt.close()

    # Calculate average MSE
    avg_mse = np.mean(errors)

    # Scatter plot: log-return volatility vs MSE
    plt.figure(figsize=(10, 6))
    plt.scatter(volatilities, errors, marker="o", color="blue")
    for i, idx in enumerate(start_indices):
        plt.text(volatilities[i], errors[i], f"Start {idx}", fontsize=9)
    plt.xlabel("Log-Return Volatility (std)")
    plt.ylabel("Mean Squared Error (MSE)")
    plt.title("Volatility vs MSE for Multiple Starting Points")
    plt.grid(True)

    # Add legend with average MSE
    plt.legend([f"Average MSE: {avg_mse:.4f}"], loc="lower right")
    plt.tight_layout()

    # Save the plot
    os.makedirs("../TSLA_pred/cnn", exist_ok=True)
    plt.savefig("../TSLA_pred/cnn/log_volatility_vs_mse.png")
    plt.show()

# Main function
def main():
    training_period = 15  # Number of days for lag features
    n_days_to_predict = 15  # Number of days to predict for each starting point
    num_start_points = 15  # Number of unique starting points
    path = "../yfinance/TSLA.csv"  # Path to your stock data file

    # Load and preprocess data
    headers = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
    df = pd.read_csv(path, names=headers, skiprows=1)
    df.replace("null", np.nan, inplace=True)
    df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]] = df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].apply(pd.to_numeric, errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'])

    # Split data into train/test sets
    X_train, y_train, X_test, y_test, test_index, feature_scaler, target_scaler = split_data(df, training_period)

    # Create Henkel matrix and decompose it
    hankel_matrix = create_henkel_matrix(df, window_size=training_period)
    trend, seasonal, noise = decompose_henkel_matrix(hankel_matrix)

    # Apply QSVD for dimensionality reduction
    u, s, vh = quantum_svd(hankel_matrix)

    # Train CNN model using the reduced components
    model = train_cnn(X_train, y_train, input_shape=(X_train.shape[1], 1))

    # Evaluate model
    evaluate_multiple_start_points(model, X_test, y_test, test_index, n_days_to_predict, num_start_points, feature_scaler, target_scaler)

if __name__ == "__main__":
    main()


Epoch 1/50


/home/mili/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.4789 - learning_rate: 0.0010
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0419 - learning_rate: 0.0010
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0204 - learning_rate: 0.0010
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0239 - learning_rate: 0.0010
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0149 - learning_rate: 0.0010
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0125 - learning_rate: 0.0010
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0153 - learning_rate: 0.0010
Epoch 8/50
21/80 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0111

KeyboardInterrupt: 